In [4]:
# !pip install netCDF4
# !pip install tqdm
import netCDF4 as nc
import numpy as np
import pandas as pd
from tqdm import tqdm

In [5]:
def read_files(year):
    months = ["01", "02", "03", '04', '05', '06', "07", '08', '09', '10', '11', '12']
    for month in months:
        path = f"./rainfalldata/COBRA_{year}-{month}_1DM_v1.0.nc"
        data = nc.Dataset(path)
        yield data, month

In [53]:
# for d, _ in read_files(2010):
#     break

In [54]:
# d

In [44]:
# # np.sum(d['precip'][:])
# time = d['time'][:].data
# p1, p2 = np.array(d['lat'][:]), np.array(d['lon'][:])
# p1 = 1.0*(p1<10) * (1.0*(0<p1))
# p1 = p1.reshape(-1, 1)
# p2 = 1.0*(p2<10) * (1.0*(0<p2))
# p2 = p2.reshape(-1, 1)
# p3 = p1 * p2.T
# # p1.shape, p2.shape, p3.shape
# np.sum(p1), np.sum(p2), np.sum(p3)

(10.0, 10.0, 100.0)

In [6]:
def get_grid(data, lat_r, lon_r):
    lats = np.array(data['lat'][:])
    lons = np.array(data['lon'][:])
    p1 = (1.0*(lat_r[0]<=lats)) * (1.0*(lats<lat_r[1]))
    p2 = (1.0*(lon_r[0]<=lons)) * (1.0*(lons<lon_r[1]))
    p1 = p1.reshape(-1, 1)
    p2 = p2.reshape(-1, 1)
    p3 = p1*p2.T
    p3 = 1.0*(p3>0)
    return p3


cols = ['year', "01", "02", "03", '04', '05', '06', "07", '08', '09', '10', '11', '12']
results = pd.DataFrame({i: [] for i in cols})
#            continent, (lat_down, lat_up), (lon_left, lon_right)
ranges = ([  'asia1',    (45,         70),    (55,         165)],
          [  'asia2',    (30,         45),    (30,         145)],
          [  'asia3',    (10,         30),    (35,         125)],
          [  'asia4',    (-10,        10),    (95,         130)],
          [  'africa1',  (5,          35),    (-20,         35)],
          [  'africa2',  (-35,         5),    (10,          40)],
          [  'europe1',  (60,         70),    (5,           55)],
          [  'europe2',  (40,         60),    (-10,         50)],
          [  'na1',      (30,         45),    (-125,       -75)],
          [  'na2',      (45,         60),    (-135,       -60)],
          [  'na3',      (15,       23.5),    (-105,       -75)],
          [  'na4',      (60,         80),    (-120,       -30)],
          [  'na5',      (60,         70),    (-165,       -70)],
          [  'na6',      (23.5,       30),    (-110,       -95)],
          [  'sa1',      (0,          10),    (-80,        -50)],
          [  'sa2',      (-20,         0),    (-80,        -40)],
          [  'sa3',      (-50,       -20),    (-75,        -65)],
          [  'o',        (-40,       -15),    (115,        150)],)

for continent, lat_r, lon_r in ranges:
    for year in range(2000, 2018):
        d = {'year': year}
        for data, month in read_files(year):
            grid = get_grid(data, lat_r, lon_r)
            rainfall = grid * np.array(data['precip'][:])
            d[f'{month}'] = np.sum(rainfall)
        results = results.append(d, ignore_index=True)

    results.to_csv(f"rain_{continent}.csv", index=False, header=False)

In [50]:
for i in ranges:
    print(i)

['asia', (0, 10), (0, 20)]
[]
